In [ ]:
import os
import sys
import urllib
import csv
import glob
import re
import numpy as np
import pandas as pd
from pandas import concat
from pandas import DataFrame
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from sklearn import datasets, linear_model
import scipy.io as sio
import datetime
import requests
from dateutil.relativedelta import relativedelta
import contextily as ctx
from matplotlib.colors import LinearSegmentedColormap
from datetime import datetime, timezone
from mintpy.utils import readfile, writefile, utils as ut
from mintpy.defaults.plot import *
from mintpy.objects.gps import search_gps, GPS
from mintpy.objects import sensor
from mintpy.view import prep_slice, plot_slice
from mintpy.cli import view, timeseries2velocity

from utils.helper_functions import is_jupyter, create_parser
from utils.helper_functions import prepend_scratchdir_if_needed, find_nearest_start_end_date
from utils.helper_functions import get_flight_direction, get_dem_extent
from utils.plot_functions import get_basemap, plot_shaded_relief
from utils.plot_functions import modify_colormap, add_colorbar
from utils.seismicity import get_earthquakes, normalize_earthquake_times
from utils.gps import get_GPS
from utils.insar import get_eos_file, generate_view_velocity_cmd, generate_view_ifgram_cmd

# %load_ext jupyter_ai
%load_ext autoreload
%autoreload 2

In [ ]:
def main(iargs=None):
    print('iargs:', iargs)
    inps = create_parser(iargs)

    print('inps:',inps)
    return inps
###########################################################################################

# if __name__ == '__main__':
if is_jupyter():
    # in Jupyter, assign command line args to sys.argv
    cmd = 'plot_data.py --help'
    cmd = 'plot_data.py MaunaLoaSenDT87 --plotType ifgram --noseismicity --nogps'
    cmd = 'plot_data.py MaunaLoaSenDT87 --plotType shaded_relief --noseismicity --nogps'
    cmd = 'plot_data.py MaunaLoaSenDT87 --plotType velocity'
    cmd = 'plot_data.py   MaunaLoaSenAT124  --plotType velocity '

    # replace multiple spaces with a single space, remove trailing space
    cmd = re.sub(' +', ' ', cmd) 
    cmd = cmd.rstrip()

    sys.argv = cmd.split(' ')
inps=main(sys.argv[1:]) 

In [ ]:
# Hardwired: move to argparse
depth_range="0 10"
cmap_name = "plasma_r"; exclude_beginning = 0.2; exclude_end = 0.2

# Hardwired for Hawaii
GPS_dir = os.getenv('SCRATCHDIR') + '/MaunaLoa/MLtry/data/'
dem_file = GPS_dir + 'demGeo.h5'  
GPS_list_file = GPS_dir + 'GPS_BenBrooks_03-05full.txt'

# get dem, earthquake and GPS data, normalize event times for plotting)
dem_shade, dem_extent = get_basemap(dem_file)

#%%capture --no-display
plot_box = inps.plot_box
start_date = inps.start_date
end_date = inps.end_date
plot_type = inps.plot_type
line_file = inps.line_file
flag_seismicity = inps.flag_seismicity
flag_gps = inps.flag_gps
gps_scale_fac = inps.gps_scale_fac
gps_key_length = inps.gps_key_length
gps_unit = inps.gps_unit

if plot_type == 'horzvert':
    plot_type='velocity'

axes=[]
if len(inps.data_dir) == 2:
    fig, axes = plt.subplots(1, 2, figsize=[12, 5] )
else:
    fig, axes = plt.subplots(figsize=[12, 5] )

i=-1
for data_dir in inps.data_dir:
    i=i+1    
    work_dir = prepend_scratchdir_if_needed(data_dir)
    if plot_type == 'velocity':
        eos_file, out_dir, vel_file = get_eos_file(work_dir)
        start_date_mod, end_date_mod = find_nearest_start_end_date(eos_file, start_date, end_date)
        start_date = start_date_mod
        end_date = end_date_mod
        cmd = f'{eos_file} --start-date {start_date} --end-date {end_date} --output {vel_file}'
        timeseries2velocity.main( cmd.split() )
            
    axes[i].set_xlim(plot_box[2], plot_box[3])
    axes[i].set_ylim(plot_box[0], plot_box[1])
    
    if plot_type == 'shaded_relief':
        plot_shaded_relief(axes[i], dem_file, plot_box = plot_box)
    else:
        if plot_type == 'velocity':
            cmd = generate_view_velocity_cmd(vel_file, plot_box)
        elif plot_type == 'ifgram':
            cmd = generate_view_ifgram_cmd(work_dir, date12, plot_box)
        data, atr, tmp_inps = prep_slice(cmd)
        axes[i], tmp_inps, im, cbar = plot_slice(axes[i], data, atr, tmp_inps)
    flight_dir = get_flight_direction(data_dir)
    axes[i].set_title(flight_dir + ': ' + start_date + ' - ' + end_date);

    # plot fault lines
    if line_file:
        lines=sio.loadmat(line_file,squeeze_me=True);
        axes[i].plot(lines['Lllh'][:,0],lines['Lllh'][:,1],color='black', linestyle='dashed',linewidth=2)
    
    # plot events
    if flag_seismicity:
        events_df = get_earthquakes(start_date, end_date, plot_box)
        norm_times = normalize_earthquake_times(events_df, start_date, end_date)
        cmap = modify_colormap( cmap_name = cmap_name, exclude_beginning = exclude_beginning, exclude_end = exclude_end, show = False)
        
        axes[i].scatter(events_df["Longitude"],events_df["Latitude"],s=2*events_df["Magnitude"] ** 3, c=norm_times,cmap=cmap,alpha=0.8)
        add_colorbar(ax = axes[i], cmap = cmap, start_date = start_date, end_date = end_date)

    if flag_gps:
        gps,lon,lat,U,V,Z,quiver_label = get_GPS(GPS_dir, GPS_list_file, plot_box, start_date, end_date, gps_unit, gps_key_length)
        quiv=axes[i].quiver(lon, lat, U, V, scale = gps_scale_fac, color='blue')
        axes[i].quiverkey(quiv, -155.50, 19.57, gps_key_length*10 , quiver_label, labelpos='N',coordinates='data',
                          color='blue',fontproperties={'size': 20}) 

plt.show()


In [ ]:
# # print(eos_file)
# atr = readfile.read_attribute(eos_file)
# mask, dict_mask = readfile.read(eos_file, datasetName="/HDFEOS/GRIDS/timeseries/quality/mask")
# # temporal_coherence, dict_temp_coh = readfile.read(eos_file, datasetName="/HDFEOS/GRIDS/timeseries/quality/temporalCoherence")

# atr['FILE_TYPE'] = 'mask'
# mask_file = vel_file.replace('velocity','maskTempCoh')
# writefile.write(mask, out_file=mask_file, metadata=atr)

# fig, ax = plt.subplots(figsize=[12, 5])
# cmd = generate_view_velocity_cmd(vel_file, mask_file, plot_box)
# #cmd = generate_view_velocity2_cmd(vel_file, mask_file, plot_box)

# print(cmd)
# data, atr, inps = prep_slice(cmd)
# ax, inps, im, cbar = plot_slice(ax, data, atr, inps)